In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.model_selection import train_test_split
import multiprocessing
import sys,os
sys.path.append(os.path.abspath("../"))
# from preprocessing.Normalize import Normalize
# import helper.SeriesHelper as series_helper
import time
import pickle
import dask.dataframe as dd
from dask.distributed import Client
import warnings
from preprocessing.Normalize import Normalize

In [2]:
warnings.filterwarnings('ignore')

In [3]:
manager = multiprocessing.Manager()
return_list = manager.list()

In [4]:

def worker(col_beg,col_end,correlation_matrix,process_count):
    '''worker function'''
    t = time.time()
    print("*****"*5,process_count," Process started","*****"*5,)
    correlated_features = []
    for i in range(col_beg,col_end,1):
        for j in range(i):
            if j!=0:
                try:
                    if abs(correlation_matrix.iloc[i, j]) > 0.8:
                        colname = correlation_matrix.columns[i]
                        correlated_features.append([colname,correlation_matrix.iloc[i, 0],correlation_matrix.iloc[i, j]])
                except Exception as e:
                    print("*****"*10,"EXCEPTION","*****"*10)
                    print("Exception :: ",e)
                    print("i,j :: ",i,j)
                    print(f'Number of rows: {len(correlation_matrix):,}.')
                    print("Number of Columns :: ",str(len(correlation_matrix.columns)))
                    print("*****"*10,"END EXCEPTION","*****"*10)
    return_list += correlated_features
    print("--"*5,"No of features to remove : ",str(len(return_list)),"--"*5)
    print("Time Required  for loop : ",time.time()-t)
    print(process_count," Chunk Process Ended")
    print("*****"*5,"  ",process_count," CHUNK END ","  ","*****"*5)

In [5]:
n = Normalize()
normal_matrix = n.get_normalized_data()
del n
del Normalize

In [6]:
corr_time = time.time()
print("Start Creating Correlation Matrix")
correlation_matrix = normal_matrix.corr()
print("Correlation Matrix Shape ::: ",correlation_matrix.shape)
del corr_time
del normal_matrix

Start Creating Correlation Matrix


In [0]:
correlation_matrix.loc[0]

In [0]:
jobs = []
step = 4000
process_count = 1

In [0]:
for i in range(0,len(correlation_matrix.columns),step):
   worker(i,i+step,correlation_matrix,process_count)

In [0]:
for x in range(0,len(return_list),1):
        print(x,"\t",x)

In [0]:
print("RETURN LIST COUNT : ",len(return_list))

In [0]:
with open('return_list.list', 'wb') as save_file: 
        # Step 3
        pickle.dump(return_list, save_file)
    print("TOTAL TIME REQURIED :: ",time.time()-t)